# Victim Analysis Notebook (SOC/IDS)

This notebook analyzes the `network_logs.csv` file generated by the attacker simulation.
It acts as a simple Intrusion Detection System (IDS) and Security Operations Center (SOC) dashboard.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the logs
try:
    df = pd.read_csv('network_logs.csv')
    print("Logs loaded successfully.")
    print(f"Total entries: {len(df)}")
except FileNotFoundError:
    print("Error: network_logs.csv not found. Please run the attacker simulation first.")

In [ ]:
# Display first few rows
df.head()

## 1. Traffic Overview

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='action', data=df)
plt.title('Distribution of Network Actions')
plt.show()

## 2. Intrusion Detection System (IDS) Logic

In [ ]:
def detect_threats(df):
    threats = []
    
    # 1. DoS Detection: High volume of requests from single IP
    request_counts = df[df['action'] == 'HTTP_REQUEST']['source_ip'].value_counts()
    for ip, count in request_counts.items():
        if count > 100:  # Threshold
            threats.append({'IP': ip, 'Type': 'DoS Attack', 'Details': f'{count} requests detected'})

    # 2. Port Scan Detection: High number of unique ports scanned
    scan_df = df[df['action'] == 'SCAN_SYN']
    port_counts = scan_df.groupby('source_ip')['port'].nunique()
    for ip, count in port_counts.items():
        if count > 10: # Threshold
            threats.append({'IP': ip, 'Type': 'Port Scanning', 'Details': f'{count} unique ports scanned'})
            
    # 3. Brute Force Detection: High number of failed logins
    login_df = df[df['action'] == 'LOGIN_ATTEMPT']
    login_counts = login_df['source_ip'].value_counts()
    for ip, count in login_counts.items():
        if count > 5: # Threshold
            threats.append({'IP': ip, 'Type': 'Brute Force', 'Details': f'{count} failed attempts'})
            
    return pd.DataFrame(threats)

threats_df = detect_threats(df)
print("Detected Threats:")
print(threats_df)

## 3. Threat Visualization

In [ ]:
if not threats_df.empty:
    plt.figure(figsize=(10, 6))
    sns.countplot(x='Type', data=threats_df, palette='Reds')
    plt.title('Detected Security Incidents')
    plt.show()
else:
    print("No threats detected.")